In [1]:
!pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.9 MB/s eta 0:00:0000:0100:01


In [9]:
import pandas as pd
import re 
import numpy as np 
import re
from tqdm import tqdm
from tqdm.notebook import tqdm
tqdm.pandas()
# import necessary packages for tfidf and statistical models: 
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# XgBoost Model for Disagreement with Ratings:

In [3]:
# read the file: 
disagreement_with_ratings_df = pd.read_csv('/Users/kartikvijay/Documents/MADS/Thesis pt.2/xgboost_LSTMS/general_training_data/disagreement_with_ratings_general.csv')
disagreement_with_ratings_df

,review_id,full_review,disagreement_with_ratings,disagreement_with_ratings_ohe
0,R1CRQ4TO7TVZYI,So the image shows noodlers polar ink series a...,0.0,0
1,R19JEPGLD3YJSH,I don’t think I have yet to write a negative r...,0.0,0
2,RITV3I7304IAG,Doesn’t erase. Even after I used some elbow gr...,0.0,0
3,R2C71Z7CM7WQ8O,Sadly I wish I had read the reviews before buy...,0.0,0
4,RGF63NTC51ASI,The media could not be loaded. ...,0.0,0
...,...,...,...,...
11225,R3P1FAGQ8FVCW6,I have never written a negative review but thi...,0.0,0
11226,R30Y7QFKEJXKU3,"Other reviewers say the same thing, so I don’t...",0.0,0
11227,RVXZJB60W7RE1,I bought based on reviews. My Epson R280 has ...,0.0,0
11228,R20ZK2HFCH5FL4,"the reviewers talk about tape, markers and bub...",0.0,0


In [4]:
def get_train_test_split(df, stratify_col, test_size=0.3, random_state=42): 
    train, test_df = train_test_split(df, test_size= test_size, random_state=random_state, stratify= stratify_col)
    return train, test_df

def create_xgb_train_data(train,test, text_col, label): 
    # tfidf - This approach uses ngrams with a range of 1,3: 
    df_train_xgb = train
    # Convert a collection of raw documents to a matrix of TF-IDF features. This is applied on the train dataset
    # ngram_range is the range for n-gram words to be taken into consideration. Here, unigram, bigram and trigram will be
    # taken into considerataion. 
    # The returned matrix of tf-idf features provides one row for each of the reviews and columns represent the terms.
    tfidf =TfidfVectorizer(ngram_range=(1,3))
    # Learn vocabulary and idf, return document-term matrix in the train dataset.
    x_train = tfidf.fit_transform(df_train_xgb[text_col])
    y_train = df_train_xgb[label]
    # need to only transform the test dataset into document-term features. Here, review-word features. Applying a 
    # transform on the transformed class
    x_test= tfidf.transform(test[text_col])
    y_test= test[label]
    return x_train, y_train, x_test, y_test

In [5]:
# train test split for disagreement with ratings df: 
train, test_df = get_train_test_split(disagreement_with_ratings_df,stratify_col=disagreement_with_ratings_df['disagreement_with_ratings_ohe']
                                       )
x_train, y_train, x_test, y_test = create_xgb_train_data(train,test_df,'full_review', 'disagreement_with_ratings_ohe')

In [ ]:
# takes a long time to run
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

param_grid = {
    'learning_rate': [0.1, 0.2, 0.3],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 0.9],
    'colsample_bytree': [0.8, 0.9],
    'n_estimators': [4000, 5000, 4500, 4100]
}

xgb = XGBClassifier(objective='binary:logistic', eval_metric='auc')
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(x_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_score_)


In [ ]:
# Testing performance metrics: 
y_pred = xgb.predict(x_test)
print(classification_report(y_pred, y_test))

In [10]:
%%time 

#Creating of the XgBoost Classifier taking input as the features returned by the tf-idf transform class from sklearn. 
# n_estimators refers to the number of trees in the classification algorithm. 
# max_depth represents the depth of each individual decision tree in the classification algrothim. Maximum split made
# in the decision trees.
xgb= xgb.XGBClassifier(n_jobs = -1, n_estimators= 4000, max_depth = 2, random_state = 32,learning_rate=0.005, gpu_id = 0
)
xgb.fit(x_train,y_train)

CPU times: user 27min 57s, sys: 49.4 s, total: 28min 46s
Wall time: 6min 3s


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.005, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=4000, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=32, ...)

In [11]:
# Testing performance metrics: 
y_pred = xgb.predict(x_test)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98      3324
           1       0.20      0.71      0.31        45

    accuracy                           0.96      3369
   macro avg       0.60      0.84      0.64      3369
weighted avg       0.99      0.96      0.97      3369



In [14]:
!pip install seaborn


  Using cached seaborn-0.12.2-py3-none-any.whl (293 kB)


In [16]:
print(cm)

[[3195   13]
 [ 129   32]]


In [19]:
%%time 

#Creating of the XgBoost Classifier taking input as the features returned by the tf-idf transform class from sklearn. 
# n_estimators refers to the number of trees in the classification algorithm.  
# max_depth represents the depth of each individual decision tree in the classification algrothim. Maximum split made
# in the decision trees.
xgb= xgb.XGBClassifier(n_jobs = -1, n_estimators= 2000, max_depth = 2, random_state = 32,learning_rate=0.005, gpu_id = 0
)
xgb.fit(x_train,y_train)

CPU times: user 11min 25s, sys: 15.6 s, total: 11min 41s
Wall time: 1min 56s


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.005, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=2000, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=32, ...)

In [20]:
# Testing performance metrics: 
y_pred = xgb.predict(x_test)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98      3344
           1       0.12      0.80      0.22        25

    accuracy                           0.96      3369
   macro avg       0.56      0.88      0.60      3369
weighted avg       0.99      0.96      0.97      3369



# XGBoost Model for Rating Management Explicit: 

In [22]:
# read the file: 
rating_management_explicit_df = pd.read_csv('/Users/kartikvijay/Documents/MADS/Thesis pt.2/xgboost/general_training_data/rating_managment_explicit_general.csv')
rating_management_explicit_df

,review_id,full_review,rating_managment_explicit,rating_managment_explicit_ohe
0,R1CRQ4TO7TVZYI,So the image shows noodlers polar ink series a...,0,0
1,R19JEPGLD3YJSH,I don’t think I have yet to write a negative r...,0,0
2,RITV3I7304IAG,Doesn’t erase. Even after I used some elbow gr...,1,1
3,R2C71Z7CM7WQ8O,Sadly I wish I had read the reviews before buy...,0,0
4,RGF63NTC51ASI,The media could not be loaded. ...,0,0
...,...,...,...,...
11225,R3P1FAGQ8FVCW6,I have never written a negative review but thi...,0,0
11226,R30Y7QFKEJXKU3,"Other reviewers say the same thing, so I don’t...",0,0
11227,RVXZJB60W7RE1,I bought based on reviews. My Epson R280 has ...,0,0
11228,R20ZK2HFCH5FL4,"the reviewers talk about tape, markers and bub...",0,0


In [26]:
# train test split for disagreement with ratings df: 
train, test_df = get_train_test_split(rating_management_explicit_df,stratify_col=rating_management_explicit_df['rating_managment_explicit_ohe']
                                       )
x_train, y_train, x_test, y_test = create_xgb_train_data(train,test_df,'full_review', 'rating_managment_explicit_ohe')

In [27]:
%%time 
#Creating of the XgBoost Classifier taking input as the features returned by the tf-idf transform class from sklearn. 
# n_estimators refers to the number of trees in the classification algorithm. 
# max_depth represents the depth of each individual decision tree in the classification algrothim. Maximum split made
# in the decision trees.
xgb= xgb.XGBClassifier(n_jobs = -1, n_estimators= 5000, max_depth = 2, random_state = 32,learning_rate=0.005, gpu_id = 0
)
xgb.fit(x_train,y_train)

CPU times: user 26min 39s, sys: 38 s, total: 27min 17s
Wall time: 4min 37s


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.005, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=5000, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=32, ...)

In [28]:
# Testing performance metrics: 
y_pred = xgb.predict(x_test)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      3357
           1       0.19      0.83      0.31        12

    accuracy                           0.99      3369
   macro avg       0.60      0.91      0.65      3369
weighted avg       1.00      0.99      0.99      3369



# XgBoost Model for Wish I had read Reviews:

In [29]:
read_reviews_df = pd.read_csv('/Users/kartikvijay/Documents/MADS/Thesis pt.2/xgboost/general_training_data/read_reviews_general.csv')
read_reviews_df

,review_id,full_review,wish_I_read_reviews,wish_I_read_reviews _ohe
0,RZQSEN3KXZJ12,Not happy with my black ink cartridge I got ma...,0,0
1,R3CL4QCWRXZEIX,I only hope that this was a quality assurance ...,0,0
2,R35NRB4TB7S6J4,I generally don't review the things I buy but ...,0,0
3,R2N024W2JDX6B0,I'm giving a single star because I have to ret...,0,0
4,R6WWRPSUQQQ97,I read a couple other reviews on how they have...,0,0
...,...,...,...,...
9855,R3P1FAGQ8FVCW6,I have never written a negative review but thi...,0,0
9856,R30Y7QFKEJXKU3,"Other reviewers say the same thing, so I don’t...",0,0
9857,RVXZJB60W7RE1,I bought based on reviews. My Epson R280 has ...,0,0
9858,R20ZK2HFCH5FL4,"the reviewers talk about tape, markers and bub...",0,0


In [33]:
# train test split for disagreement with ratings df: 
train, test_df = get_train_test_split(read_reviews_df,stratify_col=read_reviews_df['wish_I_read_reviews _ohe']
                                       )
x_train, y_train, x_test, y_test = create_xgb_train_data(train,test_df,'full_review', 'wish_I_read_reviews _ohe')

In [34]:
%%time 
#Creating of the XgBoost Classifier taking input as the features returned by the tf-idf transform class from sklearn. 
# n_estimators refers to the number of trees in the classification algorithm. 
# max_depth represents the depth of each individual decision tree in the classification algrothim. Maximum split made
# in the decision trees.
xgb= xgb.XGBClassifier(n_jobs = -1, n_estimators= 3000, max_depth = 2, random_state = 32,learning_rate=0.005, gpu_id = 0
)
xgb.fit(x_train,y_train)

CPU times: user 14min 22s, sys: 21 s, total: 14min 43s
Wall time: 2min 38s


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.005, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=3000, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=32, ...)

In [35]:
# Testing performance metrics: 
y_pred = xgb.predict(x_test)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98      2844
           1       0.52      0.71      0.60       114

    accuracy                           0.96      2958
   macro avg       0.76      0.84      0.79      2958
weighted avg       0.97      0.96      0.97      2958



#  XgBoost Model for Wrong Buying 

In [36]:
wrong_buying_df = pd.read_csv('/Users/kartikvijay/Documents/MADS/Thesis pt.2/xgboost/general_training_data/wrong_buying_general.csv')
wrong_buying_df

,review_id,full_review,wrong_buying,wrong_buying_ohe
0,R1CRQ4TO7TVZYI,So the image shows noodlers polar ink series a...,0.0,0
1,R19JEPGLD3YJSH,I don’t think I have yet to write a negative r...,0.0,0
2,RITV3I7304IAG,Doesn’t erase. Even after I used some elbow gr...,0.0,0
3,R2C71Z7CM7WQ8O,Sadly I wish I had read the reviews before buy...,0.0,0
4,RGF63NTC51ASI,The media could not be loaded. ...,0.0,0
...,...,...,...,...
11225,R3P1FAGQ8FVCW6,I have never written a negative review but thi...,0.0,0
11226,R30Y7QFKEJXKU3,"Other reviewers say the same thing, so I don’t...",0.0,0
11227,RVXZJB60W7RE1,I bought based on reviews. My Epson R280 has ...,1.0,1
11228,R20ZK2HFCH5FL4,"the reviewers talk about tape, markers and bub...",0.0,0


In [38]:
# train test split for disagreement with ratings df: 
train, test_df = get_train_test_split(wrong_buying_df,stratify_col=wrong_buying_df['wrong_buying_ohe']
                                       )
x_train, y_train, x_test, y_test = create_xgb_train_data(train,test_df,'full_review', 'wrong_buying_ohe')

In [40]:
%%time 
#Creating of the XgBoost Classifier taking input as the features returned by the tf-idf transform class from sklearn. 
# n_estimators refers to the number of trees in the classification algorithm. 
# max_depth represents the depth of each individual decision tree in the classification algrothim. Maximum split made
# in the decision trees.
xgb= xgb.XGBClassifier(n_jobs = -1, n_estimators= 6500, max_depth = 2, random_state = 32,learning_rate=0.005, gpu_id = 0
)
xgb.fit(x_train,y_train)

CPU times: user 35min 43s, sys: 49.7 s, total: 36min 33s
Wall time: 6min 17s


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.005, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=6500, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=32, ...)

In [41]:
# Testing performance metrics: 
y_pred = xgb.predict(x_test)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.98      0.94      0.96      3197
           1       0.39      0.70      0.50       172

    accuracy                           0.93      3369
   macro avg       0.69      0.82      0.73      3369
weighted avg       0.95      0.93      0.94      3369



# XGBoost for zero stars: 

In [42]:
zero_stars_df = pd.read_csv('/Users/kartikvijay/Documents/MADS/Thesis pt.2/xgboost/general_training_data/zero_star_general.csv')
zero_stars_df

,review_id,full_review,zero_star,zero_star_ohe
0,R1CRQ4TO7TVZYI,So the image shows noodlers polar ink series a...,0.0,0
1,R19JEPGLD3YJSH,I don’t think I have yet to write a negative r...,0.0,0
2,RITV3I7304IAG,Doesn’t erase. Even after I used some elbow gr...,0.0,0
3,R2C71Z7CM7WQ8O,Sadly I wish I had read the reviews before buy...,0.0,0
4,RGF63NTC51ASI,The media could not be loaded. ...,4.0,1
...,...,...,...,...
11225,R3P1FAGQ8FVCW6,I have never written a negative review but thi...,0.0,0
11226,R30Y7QFKEJXKU3,"Other reviewers say the same thing, so I don’t...",0.0,0
11227,RVXZJB60W7RE1,I bought based on reviews. My Epson R280 has ...,0.0,0
11228,R20ZK2HFCH5FL4,"the reviewers talk about tape, markers and bub...",0.0,0


In [43]:
# train test split for zero stars df: 
train, test_df = get_train_test_split(zero_stars_df,stratify_col=zero_stars_df['zero_star_ohe']
                                       )
x_train, y_train, x_test, y_test = create_xgb_train_data(train,test_df,'full_review', 'zero_star_ohe')

In [46]:
%%time 
#Creating of the XgBoost Classifier taking input as the features returned by the tf-idf transform class from sklearn. 
# n_estimators refers to the number of trees in the classification algorithm. 
# max_depth represents the depth of each individual decision tree in the classification algrothim. Maximum split made
# in the decision trees.
xgb= xgb.XGBClassifier(n_jobs = -1, n_estimators= 6500, max_depth = 2, random_state = 32,learning_rate=0.005, gpu_id = 0
)
xgb.fit(x_train,y_train)

CPU times: user 35min 8s, sys: 51 s, total: 35min 59s
Wall time: 6min 21s


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.005, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=6500, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=32, ...)

In [47]:
# Testing performance metrics: 
y_pred = xgb.predict(x_test)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98      3020
           1       0.76      0.90      0.82       349

    accuracy                           0.96      3369
   macro avg       0.87      0.93      0.90      3369
weighted avg       0.96      0.96      0.96      3369



In [48]:
print("End of Notebook")

End of Notebook
